**Import CIFAR-100 dataset**

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

class IMBALANCECIFAR10(torchvision.datasets.CIFAR10):
    cls_num = 10

    def __init__(self, root, imb_type='exp', imb_factor=0.01, rand_number=0, train=True,
                 transform=None, target_transform=None,
                 download=False):
        super(IMBALANCECIFAR10, self).__init__(root, train, transform, target_transform, download)
        np.random.seed(rand_number)
        img_num_list = self.get_img_num_per_cls(self.cls_num, imb_type, imb_factor)
        self.gen_imbalanced_data(img_num_list)

    def get_img_num_per_cls(self, cls_num, imb_type, imb_factor):
        img_max = len(self.data) / cls_num
        img_num_per_cls = []
        if imb_type == 'exp':
            for cls_idx in range(cls_num):
                num = img_max * (imb_factor**(cls_idx / (cls_num - 1.0)))
                img_num_per_cls.append(int(num))
        elif imb_type == 'step':
            for cls_idx in range(cls_num // 2):
                img_num_per_cls.append(int(img_max))
            for cls_idx in range(cls_num // 2):
                img_num_per_cls.append(int(img_max * imb_factor))
        else:
            img_num_per_cls.extend([int(img_max)] * cls_num)
        return img_num_per_cls

    def gen_imbalanced_data(self, img_num_per_cls):
        new_data = []
        new_targets = []
        targets_np = np.array(self.targets, dtype=np.int64)
        classes = np.unique(targets_np)
        # np.random.shuffle(classes)
        self.num_per_cls_dict = dict()
        for the_class, the_img_num in zip(classes, img_num_per_cls):
            self.num_per_cls_dict[the_class] = the_img_num
            idx = np.where(targets_np == the_class)[0]
            np.random.shuffle(idx)
            selec_idx = idx[:the_img_num]
            new_data.append(self.data[selec_idx, ...])
            new_targets.extend([the_class, ] * the_img_num)
        new_data = np.vstack(new_data)
        self.data = new_data
        self.targets = new_targets
        
    def get_cls_num_list(self):
        cls_num_list = []
        for i in range(self.cls_num):
            cls_num_list.append(self.num_per_cls_dict[i])
        return cls_num_list

In [45]:
def create_sampled_CIFAR10_data():

    transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

    transform_val = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

    trainset = IMBALANCECIFAR10(root='./data', train=True, download=True, transform=transform_train)
    testset = IMBALANCECIFAR10(root='./data', train=False, download=True, transform=transform_val)

    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)    
    testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

    return trainset, trainloader, testset, testloader

In [3]:
trainset, trainloader, testset, testloader = create_sampled_CIFAR10()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
def create_class_subsets(trainset, shuffle=True, batch_size=128):
    
    labels = np.array(trainset.targets)
    
    trainset_0 = torch.utils.data.Subset(trainset, list(np.where(labels == 0)[0]))
    trainset_1 = torch.utils.data.Subset(trainset, list(np.where(labels == 1)[0]))
    trainset_2 = torch.utils.data.Subset(trainset, list(np.where(labels == 2)[0]))
    trainset_3 = torch.utils.data.Subset(trainset, list(np.where(labels == 3)[0]))
    trainset_4 = torch.utils.data.Subset(trainset, list(np.where(labels == 4)[0]))
    trainset_5 = torch.utils.data.Subset(trainset, list(np.where(labels == 5)[0]))
    trainset_6 = torch.utils.data.Subset(trainset, list(np.where(labels == 6)[0]))
    trainset_7 = torch.utils.data.Subset(trainset, list(np.where(labels == 7)[0]))
    trainset_8 = torch.utils.data.Subset(trainset, list(np.where(labels == 8)[0]))
    trainset_9 = torch.utils.data.Subset(trainset, list(np.where(labels == 9)[0]))

    trainsets = [trainset_0, trainset_1, trainset_2, trainset_3, trainset_4, trainset_5, trainset_6, trainset_7, trainset_8, trainset_9]

    trainloader_0 = torch.utils.data.DataLoader(trainset_0, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_1 = torch.utils.data.DataLoader(trainset_1, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_2 = torch.utils.data.DataLoader(trainset_2, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_3 = torch.utils.data.DataLoader(trainset_3, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_4 = torch.utils.data.DataLoader(trainset_4, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_5 = torch.utils.data.DataLoader(trainset_5, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_6 = torch.utils.data.DataLoader(trainset_6, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_7 = torch.utils.data.DataLoader(trainset_7, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_8 = torch.utils.data.DataLoader(trainset_8, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    trainloader_9 = torch.utils.data.DataLoader(trainset_9, batch_size=batch_size, shuffle=shuffle, num_workers=2)

    trainloaders = [trainloader_0, trainloader_1, trainloader_2, trainloader_3, trainloader_4, trainloader_5, trainloader_6, trainloader_7, trainloader_8, trainloader_9]

    return trainsets, trainloaders


In [21]:
trainset_0, trainset_1, trainset_2, trainset_3, trainset_4, trainset_5, trainset_6, trainset_7, trainset_8, trainset_9, trainloader_0, trainloader_1, trainloader_2, trainloader_3, trainloader_4, trainloader_5, trainloader_6, trainloader_7, trainloader_8, trainloader_9 = create_class_subsets(trainset, shuffle=True, batch_size=128)
testset_0, testset_1, testset_2, testset_3, testset_4, testset_5, testset_6, testset_7, testset_8, testset_9, testloader_0, testloader_1, testloader_2, testloader_3, testloader_4, testloader_5, testloader_6, testloader_7, testloader_8, testloader_9 = create_class_subsets(testset, shuffle=False, batch_size=100)

print(len(testset_1))

599


**Creating ResNet model**

In [5]:
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from typing import Type, Any, Callable, Union, List, Optional

In [6]:
def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    '''
    Implementation is taken from the PyTorch GitHub repository
    https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
    '''

    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    '''
    Implementation is taken from the PyTorch GitHub repository
    https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
    '''    
    
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

In [7]:
class BasicBlock(nn.Module):
    '''
    Implementation is taken from the PyTorch GitHub repository
    https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
    '''
    
    expansion: int = 1

    def __init__(self, inplanes: int, planes: int, stride: int = 1, downsample: Optional[nn.Module] = None, groups: int = 1, base_width: int = 64, dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        super(BasicBlock, self).__init__()
        
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [8]:
class Bottleneck(nn.Module):
    '''
    Implementation is taken from the PyTorch GitHub repository
    https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
    '''

    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion: int = 4

    def __init__(self, inplanes: int, planes: int, stride: int = 1, downsample: Optional[nn.Module] = None, groups: int = 1, base_width: int = 64, dilation: int = 1, norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        super(Bottleneck, self).__init__()
        
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        
        width = int(planes * (base_width / 64.)) * groups
        
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

In [9]:
class ResNet(nn.Module):
    '''
    Implementation is taken from the PyTorch GitHub repository
    https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py
    '''


    def __init__(self, block: Type[Union[BasicBlock, Bottleneck]], layers: List[int], num_classes: int = 1000, zero_init_residual: bool = False, groups: int = 1, width_per_group: int = 64, replace_stride_with_dilation: Optional[List[bool]] = None, norm_layer: Optional[Callable[..., nn.Module]] = None) -> None:
        super(ResNet, self).__init__()
        
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        
        self.groups = groups
        self.base_width = width_per_group
        
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]


    def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
                    stride: int = 1, dilate: bool = False) -> nn.Sequential:
        
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        
        if dilate:
            self.dilation *= stride
            stride = 1
        
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        
        self.inplanes = planes * block.expansion
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)


    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x


    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)


In [10]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2], 10)

In [42]:
def train_congestion_avoider_10classes(trainloaders, device, model, optimizer, criterion, boolean_values, grads, epoch_counts):

    ''' 
        XXXX
    '''

    import copy

    model.train()
    start_time = time.time()

    cls_num = len(trainloaders)
    confusion_matrix = np.zeros((cls_num, cls_num))

    for epoch_count, boolean in zip(epoch_counts, boolean_values):
        if boolean:
            epoch_count = 0
    
    for cls_num, trainloader in enumerate(trainloaders):
        for batch_idx, (inputs, targets) in enumerate(trainloader):
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
        
            outputs = model(inputs)
            loss = criterion(outputs,targets)
        
            # Back-propagate the loss due to 'cats'
            loss.backward(retain_graph=True)
            with torch.no_grad():
                for name, parameter in model.named_parameters():
                  try:
                      if name not in grads[cls_num].keys():
                          grads[cls_num][name] = torch.mul(copy.deepcopy(parameter.grad), optimizer.param_groups[0]['lr'])
                      else:
                          grads[cls_num][name] += torch.mul(copy.deepcopy(parameter.grad), optimizer.param_groups[0]['lr'])
                  except:
                      pass
            optimizer.step()

            _, predicted = outputs.max(1)

            for target, pred in zip(targets, predicted):
                confusion_matrix[target][pred] += 1

    accuracies = np.zeros((10))
    recalls = np.zeros((10))
    precisions = np.zeros((10))
    fScores = np.zeros((10))

    for epoch_count in epoch_counts:
      epoch_count += 1

    for cls in range(cls_num):
      accuracies[cls] = confusion_matrix[cls][cls] / confusion_matrix.sum()
      recalls[cls] = confusion_matrix[cls][cls] / confusion_matrix.sum(0)[cls]
      precisions[cls] = confusion_matrix[cls][cls] / confusion_matrix.sum(1)[cls]
      try:
        fScores[cls] = 2 * precisions[cls] * recalls[cls] / (precisions[cls] + recalls[cls])
      except:
        fScores[cls] = 0

    return confusion_matrix, accuracies, recalls, precisions, fScores, grads, epoch_counts

In [28]:
def linear_cong_condition(min_cond, max_cond, epoch, max_epochs):

    condition = min_cond + (max_cond - min_cond) * (epoch / max_epochs)

    return condition

In [30]:
def congestion_avoid(model, optimizer, metrics, condition, grads, min_epochs, mult, epoch_counts):
    '''
    global epoch_count_one
    global epoch_count_two

    boolean_one = False
    boolean_two = False

    branch1_cond = (branch1_metric < condition * branch2_metric) and (epoch_count_two >= min_epochs)
    branch2_cond = (branch2_metric < condition * branch1_metric) and (epoch_count_one >= min_epochs)

    if branch1_cond:
        boolean_one = True
        print('Branch 1 condition has been met ..... {:.2f}%'.format(100.*condition))
        for name, value in model.named_parameters():
            with torch.no_grad():
                if name in branch_two_grads.keys():
                    value += mult * branch_two_grads[name]
        for name in branch_two_grads.keys():
            branch_two_grads[name] -= mult * branch_two_grads[name]
        epoch_count_two = 0

    elif branch2_cond:
        boolean_two = True
        print('Branch 2 condition has been met ..... {:.2f}%'.format(100.*condition))
        for name, value in model.named_parameters():
            with torch.no_grad():
                if name in branch_one_grads.keys():
                    value += mult * branch_one_grads[name]
        for name in branch_one_grads.keys():
            branch_one_grads[name] -= mult * branch_one_grads[name]
        epoch_count_one = 0
    
    else:
        print('No condition is met ..... {:.2f}%'.format(100.*condition))'''

    return

In [43]:
def test_congestion_avoider_10classes(start_time, testloaders, device, model, optimizer, scheduler, grads, criterion, epoch, max_epochs, min_cond, max_cond, min_epochs, mult, epoch_counts):

    ''' 
        XXXX
    '''

    import copy

    model.eval()
    cls_num = len(testloaders)
    confusion_matrix = np.zeros((cls_num, cls_num))
    accuracies = np.zeros((10))
    recalls = np.zeros((10))
    precisions = np.zeros((10))
    fScores = np.zeros((10))
    boolean_values = np.zeros((10))
    
    with torch.no_grad():
        for cls_num, testloader in enumerate(testloaders):
            for batch_idx, (inputs, targets) in enumerate(testloader):
                inputs = inputs.to(device)
                targets = targets.to(device)
            
                outputs = model(inputs)
                loss = criterion(outputs,targets)
                _, predicted = outputs.max(1)

                for target, pred in zip(targets, predicted):
                    confusion_matrix[target][pred] += 1
    
        for cls in range(cls_num):
            accuracies[cls] = confusion_matrix[cls][cls] / confusion_matrix.sum() 
            recalls[cls] = confusion_matrix[cls][cls] / confusion_matrix.sum(0)[cls]
            precisions[cls] = confusion_matrix[cls][cls] / confusion_matrix.sum(1)[cls]
            try:
                fScores[cls] = 2 * precisions[cls] * recalls[cls] / (precisions[cls] + recalls[cls])
            except:
                fScores[cls] = 0

        condition = linear_cong_condition(min_cond, max_cond, epoch, max_epochs)
        congestion_avoid(model, optimizer, precisions, condition, grads, min_epochs, mult, epoch_counts)
        scheduler.step()

        print('time: %.3f sec'% ((time.time()-start_time)))
        print(confusion_matrix)
        for cls in range(cls_num):
            print('Class %d A: : %.3f%% (%d/%d)'%(cls, 100*accuracies[cls], confusion_matrix[cls][cls], confusion_matrix.sum()))
            print('Class %d P: : %.3f%% (%d/%d)'%(cls, 100*precisions[cls], confusion_matrix[cls][cls], confusion_matrix.sum(1)[cls]))
            print('Class %d R: : %.3f%% (%d/%d)'%(cls, 100*recalls[cls], confusion_matrix[cls][cls], confusion_matrix.sum(0)[cls]))
            print('Class %d F: : %.3f%%'%(cls, 100*fScores[cls]))
            print('********************')


    return optimizer, accuracies, precisions, recalls, fScores, boolean_values, grads, epoch_counts

In [46]:
import torch.optim as optim
from torch.optim.lr_scheduler import CyclicLR
import time
import torch.backends.cudnn as cudnn

def get_cong_avoidance_results_10classes(epochs=100, min_cond=0.95, max_cond = 0.99, mult=1, lr=0.1, min_epochs = 5):

    '''Allow the congestion condition to change linearly over time '''

    # Create ResNet model
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = ResNet18()
    model = model.to(device)
    if device == 'cuda':
        print('CUDA device used...')
        model = torch.nn.DataParallel(model)
        cudnn.benchmark = True

    # Import data
    trainset, trainloader, testset, testloader = create_sampled_CIFAR10_data()
    trainsets, trainloaders = create_class_subsets(trainset, shuffle=True, batch_size=128)
    testsets, testloaders = create_class_subsets(testset, shuffle=False, batch_size=100)

    # Create variables
    cls_num = len(trainset.classes)
    boolean_values = [False]*cls_num
    grads = [{}]*cls_num
    epoch_counts = [0]*cls_num
    
    criterion = nn.CrossEntropyLoss()
    # CREATE MODEL OPTIMIZER
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0, weight_decay=5e-4)
    scheduler = CyclicLR(optimizer, base_lr=0.0001, max_lr=lr, step_size_up=10, mode="triangular2")

    # BEGIN RECORDING THE TIME
    start_time = time.time()

    # Create matrices to store results

    train_acc = np.zeros((epochs, cls_num))
    train_P = np.zeros((epochs, cls_num))
    train_R = np.zeros((epochs, cls_num))
    train_F = np.zeros((epochs, cls_num))
    test_acc = np.zeros((epochs, cls_num))
    test_P = np.zeros((epochs, cls_num))
    test_R = np.zeros((epochs, cls_num))
    test_F = np.zeros((epochs, cls_num))
    cong_events = np.zeros((epochs, cls_num))

    for epoch in range(epochs):
        print('\n********** EPOCH {} **********'.format(epoch + 1))
        print('Learning rate: ', optimizer.param_groups[0]['lr'])
        confusion_matrix, accuracies, recalls, precisions, fScores, grads, epoch_counts = train_congestion_avoider_10classes(trainloaders, device, model, optimizer, criterion, boolean_values, grads, epoch_counts)
        train_acc[epoch] = accuracies
        train_P[epoch] = precisions
        train_R[epoch] = recalls
        train_F[epoch] = fScores
        optimizer, accuracies, precisions, recalls, fScores, boolean_values, grads, epoch_counts = test_congestion_avoider_10classes(start_time, testloaders, device, model, optimizer, scheduler, grads, criterion, epoch, epochs, min_cond, max_cond, min_epochs, mult, epoch_counts)
        test_acc[epoch] = accuracies
        test_P[epoch] = precisions
        test_R[epoch] = recalls
        test_F[epoch] = fScores
        cong_events[epoch] = boolean_values

    return train_acc, train_P, train_R, train_F, test_acc, test_P, test_R, test_F, cong_events

In [44]:
get_cong_avoidance_results_10classes(epochs=1, min_cond=0, max_cond = 0, mult=0, lr=0.1, min_epochs = 5)

CUDA device used...
Files already downloaded and verified
Files already downloaded and verified

********** EPOCH 1 **********
Learning rate:  0.0001


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


time: 7.951 sec
[[  2.   6. 372. 475. 137.   8.   0.   0.   0.   0.]
 [  1.   2. 245. 268.  74.   7.   1.   1.   0.   0.]
 [  0.   2. 163. 148.  42.   4.   0.   0.   0.   0.]
 [  0.   2.  98.  89.  25.   1.   0.   0.   0.   0.]
 [  1.   2.  65.  43.  18.   0.   0.   0.   0.   0.]
 [  0.   1.  24.  37.  14.   1.   0.   0.   0.   0.]
 [  0.   0.  18.  21.   5.   2.   0.   0.   0.   0.]
 [  0.   0.  12.  10.   4.   1.   0.   0.   0.   0.]
 [  1.   0.   5.   9.   1.   0.   0.   0.   0.   0.]
 [  0.   0.   5.   5.   0.   0.   0.   0.   0.   0.]]
Class 0 A: : 0.081% (2/2478)
Class 0 P: : 0.200% (2/1000)
Class 0 R: : 40.000% (2/5)
Class 0 F: : 0.398%
********************
Class 1 A: : 0.081% (2/2478)
Class 1 P: : 0.334% (2/599)
Class 1 R: : 13.333% (2/15)
Class 1 F: : 0.651%
********************
Class 2 A: : 6.578% (163/2478)
Class 2 P: : 45.404% (163/359)
Class 2 R: : 16.187% (163/1007)
Class 2 F: : 23.865%
********************
Class 3 A: : 3.592% (89/2478)
Class 3 P: : 41.395% (89/215)
Class

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in double_scalars


(array([[0.31984524, 0.06714493, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]]),
 array([[0.7936    , 0.27794461, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]]),
 array([[0.64699168, 0.23786408, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]]),
 array([[0.71283571, 0.25634713,        nan,        nan,        nan,
                nan,        nan,        nan,        nan, 0.        ]]),
 array([[0.0008071 , 0.0008071 , 0.06577885, 0.03591606, 0.00726392,
         0.00040355, 0.        , 0.        , 0.        , 0.        ]]),
 array([[0.002     , 0.0033389 , 0.454039  , 0.41395349, 0.13953488,
         0.01298701, 0.        , 0.        , 0.        , 0.        ]]),
 array([[0.4       , 0.13333333, 0.16186693, 0.08054299, 0.05625   ,
         0.04166667, 0.        , 0.        ,        nan, 0.        ]]),
 array([[0.00